In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from sweref99 import projections
from datetime import datetime, date

In [2]:
rpath='./data/Brandriskdata 2000-2020.csv'
fpath='./data/Insatser till brand i skog och mark 2000-2020.xlsx'
#'PunktID': str, 'E': str, 'N': str,
rtypes={ 'PunktID': str}

In [3]:
#Reading csv file from path with dtypes
risk = pd.read_csv(rpath, sep=';', dtype=rtypes, )
fires = pd.read_excel(fpath)

In [4]:
fireRisk = risk
reportedFires = fires

In [5]:
fireRisk[['E','N']] = fireRisk[['E','N']].fillna(0)
fireRisk['N'] = fireRisk['N'].astype(int)
fireRisk['E'] = fireRisk['E'].astype(int)


In [6]:
print(fireRisk.isnull().sum(axis = 0))
print("Total rows: ")
print(fireRisk.shape[0])

PunktID                 0
E                       0
N                       0
Kommun                  2
Datum                   2
Temp                14617
Tmedel              38049
Nederbord           23520
RH                  33459
Vindhastighet        3991
Vindriktning     12310774
FFMC              1701172
DMC               1701172
DC                1701172
ISI               1701172
BUI               1701172
FWI               1701172
FWI_index         1701172
HBV_o             1404342
HBV_u             1404342
HBV               1404342
HBV_index         1404342
Gras              6855866
dtype: int64
Total rows: 
19188859


In [7]:
fireRisk.dropna(subset=['FWI'], how='all', inplace=True)
fireRisk.dropna(subset=['Tmedel'], how='all', inplace=True)


In [8]:
print(fireRisk.isnull().sum(axis = 0))
print("Total rows: ")
print(fireRisk.shape[0])

PunktID                 0
E                       0
N                       0
Kommun                  0
Datum                   0
Temp                    0
Tmedel                  0
Nederbord               0
RH                   3989
Vindhastighet           0
Vindriktning     11068726
FFMC                    0
DMC                     0
DC                      0
ISI                     0
BUI                     0
FWI                     0
FWI_index               0
HBV_o               39890
HBV_u               39890
HBV                 39890
HBV_index           39890
Gras              6777056
dtype: int64
Total rows: 
17474853


In [9]:
#Function that calculates number of missing data in column of dataframe and prints result.
def missing(df,column):
       x = len(df)
       if df[column].isnull().any():
           print('{0} has total of {1} null values'.format(column, df[column].isnull().sum()))
           print ('In the column {0}'.format(column), round(100-(df[column].count()/x * 100), 3), '% of the cells have missing values')
    

In [10]:
missing(fireRisk,'FWI')

In [11]:
fireRisk.columns

Index(['PunktID', 'E', 'N', 'Kommun', 'Datum', 'Temp', 'Tmedel', 'Nederbord',
       'RH', 'Vindhastighet', 'Vindriktning', 'FFMC', 'DMC', 'DC', 'ISI',
       'BUI', 'FWI', 'FWI_index', 'HBV_o', 'HBV_u', 'HBV', 'HBV_index',
       'Gras'],
      dtype='object')

In [12]:
fireRisk = fireRisk.drop(columns=['Nederbord',
       'RH', 'Vindhastighet', 'Vindriktning', 'FFMC', 'DMC', 'DC', 'ISI',
       'BUI', 'HBV_o', 'HBV_u', 'HBV', 'HBV_index',
       'Gras'])
fireRisk

,PunktID,E,N,Kommun,Datum,Temp,Tmedel,FWI,FWI_index
0,1,454334,6690301,2021.0,2000-04-01,"1,8","1,6",0,1.0
1,1,454334,6690301,2021.0,2000-04-02,"-2,3","-2,3",0,1.0
2,1,454334,6690301,2021.0,2000-04-03,"-1,4",-3,0,1.0
3,1,454334,6690301,2021.0,2000-04-04,"0,1","-0,2",0,1.0
4,1,454334,6690301,2021.0,2000-04-05,"0,4","-2,8",0,1.0
...,...,...,...,...,...,...,...,...,...
19188852,7892,375482,6733336,1737.0,2020-12-27,"0,4",0,0,-1.0
19188853,7892,375482,6733336,1737.0,2020-12-28,"-0,1",0,0,-1.0
19188854,7892,375482,6733336,1737.0,2020-12-29,"0,3",0,0,-1.0
19188855,7892,375482,6733336,1737.0,2020-12-30,"-0,1",0,0,-1.0


In [13]:
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [14]:
#Filter rows on cause of fire and only keeping "Annan". "Okänd", "Grillning eller lägereld" and "Uppgift saknas"
copy_df = filter_rows_by_values(reportedFires, 'BEJBbrandorsakText', ['Fyrverkeri eller pyroteknik', 'Annan',
       'Barns lek med eld', 'Annan eldning', 'Avsiktlig brand',
       'Övriga gnistor',
       'Återantändning av brand från tidigare räddningsinsats',
       'Självantändning - biologisk eller kemisk', 'Blixtnedslag', 'Heta arbeten',
       'Rökning', 'Tågbromsning', 'Fel i utrustning', 'Uppgift saknas'])

In [15]:
fireRisk.dtypes

PunktID       object
E              int32
N              int32
Kommun       float64
Datum         object
Temp          object
Tmedel        object
FWI           object
FWI_index    float64
dtype: object

In [16]:
fireRisk['FWI'] = fireRisk['FWI'].str.replace(',','.')
fireRisk['Temp'] = fireRisk['Temp'].str.replace(',','.')
fireRisk['Tmedel'] = fireRisk['Tmedel'].str.replace(',','.')

In [17]:
fireRisk['FWI'] = fireRisk['FWI'].astype(float)
fireRisk['Temp'] = fireRisk['Temp'].astype(float)
fireRisk['Tmedel'] = fireRisk['Tmedel'].astype(float)


In [18]:
fireRisk['Kommun'] = fireRisk['Kommun'].astype(int)
fireRisk['Datum'] = pd.to_datetime(fireRisk['Datum'], format='%Y-%m-%d')

In [19]:
fireRiskT = fireRisk.drop(columns=['E','N'])
fireRiskT

,PunktID,Kommun,Datum,Temp,Tmedel,FWI,FWI_index
0,1,2021,2000-04-01,1.8,1.6,0.0,1.0
1,1,2021,2000-04-02,-2.3,-2.3,0.0,1.0
2,1,2021,2000-04-03,-1.4,-3.0,0.0,1.0
3,1,2021,2000-04-04,0.1,-0.2,0.0,1.0
4,1,2021,2000-04-05,0.4,-2.8,0.0,1.0
...,...,...,...,...,...,...,...
19188852,7892,1737,2020-12-27,0.4,0.0,0.0,-1.0
19188853,7892,1737,2020-12-28,-0.1,0.0,0.0,-1.0
19188854,7892,1737,2020-12-29,0.3,0.0,0.0,-1.0
19188855,7892,1737,2020-12-30,-0.1,0.0,0.0,-1.0


In [20]:
test = fireRiskT.groupby(['Datum','Kommun']).mean()
#test = fireRiskT.groupby(['Datum','Kommun'])['FWI_index'].mean()
#test = test.reset_index()
test


Temp    Tmedel      FWI  FWI_index
Datum      Kommun                                        
2000-04-01 115     3.450000  0.475000  5.25000       2.00
           117     2.980000  1.340000  4.44000       2.00
           120     2.450000  1.775000  2.32500       1.25
           125     3.900000  1.725000  5.32500       2.00
           126     3.700000  1.800000  4.70000       2.00
...                     ...       ...      ...        ...
2020-12-31 2580    0.690909 -2.454545  0.00000      -1.00
           2581   -3.681818 -4.545455  0.00000      -1.00
           2582   -2.157143 -3.942857  0.00000      -1.00
           2583    0.892308 -1.538462  0.00000      -1.00
           2584   -5.731250 -6.000000  0.00625      -1.00

[1236852 rows x 4 columns]

In [23]:
test = test.reset_index()
test = test.rename(columns={"Datum": "datum", "Kommun":"kommun"})


In [30]:
merged = pd.merge(reportedFires, test, how='inner', on=['kommun','datum'])
merged

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,index,Temp,Tmedel,FWI,FWI_index
0,2000,2000-04-01,12:52:00,980,Gotland,Verksamhet inte knuten till en byggnad,6381604.0,694625.0,Annan eldning,2000,0,2500,79,7.369811,3.45283,1.373585,1.415094
1,2000,2000-04-01,12:57:00,980,Gotland,Verksamhet inte knuten till en byggnad,6385532.0,688580.0,Okänd,20000,0,0,79,7.369811,3.45283,1.373585,1.415094
2,2000,2000-04-01,14:22:00,980,Gotland,Verksamhet inte knuten till en byggnad,6389637.0,697530.0,Okänd,0,0,120,79,7.369811,3.45283,1.373585,1.415094
3,2000,2000-04-01,13:35:00,182,Nacka,Verksamhet inte knuten till en byggnad,NaN,NaN,Annan eldning,0,0,30,13,3.500000,1.90000,4.500000,2.000000
4,2000,2000-04-01,13:50:00,1763,Forshaga,Verksamhet inte knuten till en byggnad,6616172.0,414889.0,Annan eldning,0,0,7000,178,4.950000,3.17500,5.950000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86261,2020,2020-12-29,01:52:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6586007.0,664319.0,Avsiktlig brand,0,1,0,1236023,5.250000,4.00000,0.000000,-1.000000
86262,2020,2020-12-29,18:56:00,117,Österåker,Verksamhet inte knuten till en byggnad,6599203.0,687136.0,Avsiktlig brand,0,0,1,1236013,4.625000,4.00000,0.000000,-1.000000
86263,2020,2020-12-30,12:32:00,380,Uppsala,Verksamhet inte knuten till en byggnad,6637240.0,653312.0,Okänd,0,0,20,1236315,1.036842,2.00000,0.000000,-1.000000
86264,2020,2020-12-31,15:17:00,1384,Kungsbacka,Verksamhet inte knuten till en byggnad,6360762.0,327099.0,Annan,0,0,1,1236696,4.033333,3.50000,0.000000,-1.000000


In [31]:
merged = merged.drop(columns='index')


In [32]:
merged

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Temp,Tmedel,FWI,FWI_index
0,2000,2000-04-01,12:52:00,980,Gotland,Verksamhet inte knuten till en byggnad,6381604.0,694625.0,Annan eldning,2000,0,2500,7.369811,3.45283,1.373585,1.415094
1,2000,2000-04-01,12:57:00,980,Gotland,Verksamhet inte knuten till en byggnad,6385532.0,688580.0,Okänd,20000,0,0,7.369811,3.45283,1.373585,1.415094
2,2000,2000-04-01,14:22:00,980,Gotland,Verksamhet inte knuten till en byggnad,6389637.0,697530.0,Okänd,0,0,120,7.369811,3.45283,1.373585,1.415094
3,2000,2000-04-01,13:35:00,182,Nacka,Verksamhet inte knuten till en byggnad,NaN,NaN,Annan eldning,0,0,30,3.500000,1.90000,4.500000,2.000000
4,2000,2000-04-01,13:50:00,1763,Forshaga,Verksamhet inte knuten till en byggnad,6616172.0,414889.0,Annan eldning,0,0,7000,4.950000,3.17500,5.950000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86261,2020,2020-12-29,01:52:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6586007.0,664319.0,Avsiktlig brand,0,1,0,5.250000,4.00000,0.000000,-1.000000
86262,2020,2020-12-29,18:56:00,117,Österåker,Verksamhet inte knuten till en byggnad,6599203.0,687136.0,Avsiktlig brand,0,0,1,4.625000,4.00000,0.000000,-1.000000
86263,2020,2020-12-30,12:32:00,380,Uppsala,Verksamhet inte knuten till en byggnad,6637240.0,653312.0,Okänd,0,0,20,1.036842,2.00000,0.000000,-1.000000
86264,2020,2020-12-31,15:17:00,1384,Kungsbacka,Verksamhet inte knuten till en byggnad,6360762.0,327099.0,Annan,0,0,1,4.033333,3.50000,0.000000,-1.000000


In [35]:
above4 = merged[merged['FWI_index']>4]
above4

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Temp,Tmedel,FWI,FWI_index
1150,2000,2000-05-03,18:50:00,881,Nybro,Verksamhet inte knuten till en byggnad,6311333.0,560686.0,Annan,150,0,0,20.412500,9.875000,20.662500,4.250000
1420,2000,2000-05-07,13:16:00,380,Uppsala,Verksamhet inte knuten till en byggnad,6635996.0,647573.0,Okänd,0,0,100,18.200000,10.047368,21.178947,4.368421
1474,2000,2000-05-08,10:12:00,881,Nybro,Verksamhet inte knuten till en byggnad,6283807.0,540816.0,Övriga gnistor,10000,0,0,23.025000,12.800000,20.637500,4.250000
1475,2000,2000-05-08,10:12:00,881,Nybro,Verksamhet inte knuten till en byggnad,6283807.0,540816.0,Övriga gnistor,10000,0,0,23.025000,12.800000,20.637500,4.250000
1528,2000,2000-05-08,21:15:00,1281,Lund,Verksamhet inte knuten till en byggnad,6174043.0,386165.0,Avsiktlig brand,0,0,5,20.075000,11.750000,23.725000,4.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85651,2020,2020-08-27,20:38:00,192,Nynäshamn,Verksamhet inte knuten till en byggnad,6535829.0,667695.0,Annan eldning,0,0,10,19.280000,15.000000,22.080000,4.400000
85653,2020,2020-08-27,22:21:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6572544.0,673272.0,Okänd,0,10,0,19.550000,16.000000,27.550000,5.500000
85667,2020,2020-08-28,14:57:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6609656.0,589536.0,Okänd,0,0,3,17.922222,11.666667,24.066667,4.777778
85668,2020,2020-08-28,15:22:00,1885,Lindesberg,Verksamhet inte knuten till en byggnad,6612185.0,502645.0,Okänd,0,1,0,17.166667,11.833333,21.583333,4.416667


In [36]:
above4 = filter_rows_by_values(above4, 'BEJBbrandorsakText', ['Fyrverkeri eller pyroteknik', 'Annan',
       'Barns lek med eld', 'Annan eldning', 'Avsiktlig brand',
       'Övriga gnistor',
       'Återantändning av brand från tidigare räddningsinsats',
       'Självantändning - biologisk eller kemisk', 'Blixtnedslag', 'Heta arbeten',
       'Rökning', 'Tågbromsning', 'Fel i utrustning', 'Uppgift saknas'])
above4

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Temp,Tmedel,FWI,FWI_index
1420,2000,2000-05-07,13:16:00,380,Uppsala,Verksamhet inte knuten till en byggnad,6635996.0,647573.0,Okänd,0,0,100,18.200000,10.047368,21.178947,4.368421
1532,2000,2000-05-09,01:30:00,1980,Västerås,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,0,100,22.366667,15.066667,20.300000,4.111111
1533,2000,2000-05-09,09:09:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6607977.0,582930.0,Okänd,0,0,1,22.366667,15.066667,20.300000,4.111111
1534,2000,2000-05-09,21:47:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6609205.0,585442.0,Okänd,0,0,1,22.366667,15.066667,20.300000,4.111111
1539,2000,2000-05-09,08:03:00,180,Stockholm,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,8,0,22.900000,14.650000,22.450000,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85641,2020,2020-08-27,16:35:00,1983,Köping,Verksamhet inte knuten till en byggnad,6593620.0,563258.0,Okänd,70,0,0,19.350000,12.833333,26.483333,5.166667
85644,2020,2020-08-27,16:53:00,140,Nykvarn,Verksamhet inte knuten till en byggnad,6557858.0,642314.0,Okänd,0,30,0,19.233333,14.333333,25.333333,5.000000
85653,2020,2020-08-27,22:21:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6572544.0,673272.0,Okänd,0,10,0,19.550000,16.000000,27.550000,5.500000
85667,2020,2020-08-28,14:57:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6609656.0,589536.0,Okänd,0,0,3,17.922222,11.666667,24.066667,4.777778
